In [1]:
import numpy as np
import tensorflow as tf

In [2]:
np.random.seed(1793)
tf.random.set_seed(1793)

In [3]:
import trieste
from trieste.utils.objectives import branin

search_space = trieste.space.Box([0, 0], [1, 1])

In [4]:
def new_branin(data):
    print(data.shape)
    return branin(data)

In [5]:
from trieste.acquisition.rule import OBJECTIVE

observer = trieste.utils.objectives.mk_observer(new_branin, OBJECTIVE)

num_initial_points = 5
initial_query_points = search_space.sample(num_initial_points)
initial_data = observer(initial_query_points)

(5, 2)


In [6]:
import gpflow

def build_model(data):
    print("Building a model")
    variance = tf.math.reduce_variance(data.observations)
    kernel = gpflow.kernels.Matern52(variance=variance, lengthscales=[0.2, 0.2])
    gpr = gpflow.models.GPR(data.astuple(), kernel, noise_variance=1e-5)
    gpflow.set_trainable(gpr.likelihood, False)

    return {OBJECTIVE: {
        "model": gpr,
        "optimizer": gpflow.optimizers.Scipy(),
        "optimizer_args": {
            "minimize_args": {"options": dict(maxiter=100)},
        },
    }}

model = build_model(initial_data[OBJECTIVE])

Building a model


In [7]:
bo = trieste.bayesian_optimizer.BayesianOptimizer(observer, search_space)

result = bo.optimize(25, initial_data, model)
dataset = result.try_get_final_datasets()[OBJECTIVE]

(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
Optimization completed without errors


In [8]:
query_points = dataset.query_points.numpy()
observations = dataset.observations.numpy()

arg_min_idx = tf.squeeze(tf.argmin(observations, axis=0))

print(f"query point: {query_points[arg_min_idx, :]}")
print(f"observation: {observations[arg_min_idx, :]}")

query point: [0.54162649 0.14935401]
observation: [0.40162417]


In [12]:
initial_data[OBJECTIVE].observations

<tf.Tensor: shape=(5, 1), dtype=float64, numpy=
array([[ 52.74206955],
       [ 20.91833473],
       [110.23393455],
       [225.05283879],
       [ 52.49971937]])>